# Dataset 확인

## 기본 영상 길이 넘는지 확인

In [84]:
import cv2
import os
import pandas as pd

person_0 = []
person_1 = []
person_2 = []
person_3 = []
person_4 = []

data_dir = 'korea_data'
data_per = os.listdir(data_dir)
for person in data_per:
    words_dir= f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for id in Id:
        if id == '.DS_Store':
            continue
        id_dir = f'{data_dir}/{person}/{id}'
        num = os.listdir(id_dir)
        if len(num) == 3:
            num = os.listdir(id_dir)
            for n  in num:
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}/image'
                frames = os.listdir(words_dir_2)
                if frames == '.DS_Store':
                    continue
                if len(frames) < 5:
                    if person == '00':
                        person_0.append((id, n))
                    elif person == '01':
                        person_1.append((id, n))
                    elif person == '02':
                        person_2.append((id, n))
                    elif person == '03':
                        person_3.append((id, n))
                    else:
                        person_4.append((id, n))
print(len(person_0),len(person_1),len(person_2),len(person_3),len(person_4))

0 0 0 0 0


## 데이터셋 전처리(crop)
- 입술 주변을 Detect 후 100x50 pixel crop

In [1]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((20, 2), dtype=dtype)
    # for only lip landmarks
    n = 0
    for i in range(48, shape.num_parts):
        coords[n] = (shape.part(i).x, shape.part(i).y)
        n += 1

    # return the list of (x, y)-coordinates
    return coords

In [25]:
faces = f'korea_data/00/11/00/image'
for jpg in os.listdir(faces):
    if jpg[3:6] not in ['_0.', '_1.', '_2.', '_3.', '_4.']:
        pass
    else:
        dir = f'{faces}/{i}'
        src = cv2.imread(dir, cv2.IMREAD_COLOR)
        img_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        face_detector = detector(img_gray, 1)
        for face in face_detector:
            landmarks = predictor(src, face)
            landmarks = shape_to_np(landmarks)
            print("gotfaces!")

gotfaces!
gotfaces!
gotfaces!
gotfaces!
gotfaces!


In [26]:
## Dataset crop Demo
from keras.preprocessing import image
import cv2
import os
import pandas as pd
import dlib
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") # predict 해주는애            

data_dir = 'korea_data'
data_per = os.listdir(data_dir)
cnt = 0
for person in data_per:
    for sentence in os.listdir(f'{data_dir}/{person}' ):
        if sentence == 'csv' or sentence == 'video' or sentence == '.DS_Store':
            continue
        for nums in os.listdir(f'{data_dir}/{person}/{sentence}'):
            if nums == '.DS_Store':
                continue
            count = 0
            print(f'{data_dir}/{person}/{sentence}/{nums}/image')
            for jpg in os.listdir(f'{data_dir}/{person}/{sentence}/{nums}/image'):
                # 3초 안으로
                if jpg[3:6] not in ['_0.', '_1.', '_2.', '_3.', '_4.']:
                    pass
                else:
                    faces = f'{data_dir}/{person}/{sentence}/{nums}/image/{jpg}'
                    src = cv2.imread(faces, cv2.IMREAD_COLOR)
                    img_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
                    face_detector = detector(img_gray, 1)
                    for face in face_detector:
                        landmarks = predictor(src, face)
                        landmarks = shape_to_np(landmarks)
                        # select center of mouth
                        x_list = [x[0] for x in landmarks]
                        y_list = [y[1] for y in landmarks]
                        x = sum(x_list)//20
                        y = sum(y_list)//20
                    dst = src[y-50:y+50, x-100:x+100].copy()
                    # 확인용
                    #cv2.imshow("src", src)
                    #cv2.imshow("dst", dst)
                    #cv2.waitKey()
                    #cv2.destroyAllWindows()
                    img_tensor = image.img_to_array(dst) #tensor로 type 변경
                    img_tensor /= 255. # scaling  # (100, 200, 3)
                    img_tensor = np.expand_dims(img_tensor, axis = 0) #(1, 100, 200, 3)
                    # frame 별 묶기
                    if count == 0:
                        x_data = img_tensor
                        count += 1
                    else:
                        x_data = np.concatenate((x_data, img_tensor), axis = 0)
            # (1, 5, 100, 200, 3)
            x_data = np.expand_dims(x_data, axis = 0) 
            print("x_data:", x_data.shape)
            if cnt == 0:
                x_data_ = x_data
                cnt += 1
            else:
                x_data_ = np.concatenate((x_data_, x_data), axis = 0)
            print(x_data_.shape)
                #(갯수, frame, 64, 64, 3)

korea_data/00/00/00/image
x_data: (1, 5, 100, 200, 3)
(1, 5, 100, 200, 3)
korea_data/00/00/01/image
x_data: (1, 5, 100, 200, 3)
(2, 5, 100, 200, 3)
korea_data/00/00/02/image
x_data: (1, 5, 100, 200, 3)
(3, 5, 100, 200, 3)
korea_data/00/01/00/image
x_data: (1, 5, 100, 200, 3)
(4, 5, 100, 200, 3)
korea_data/00/01/01/image
x_data: (1, 5, 100, 200, 3)
(5, 5, 100, 200, 3)
korea_data/00/01/02/image
x_data: (1, 5, 100, 200, 3)
(6, 5, 100, 200, 3)
korea_data/00/02/00/image
x_data: (1, 5, 100, 200, 3)
(7, 5, 100, 200, 3)
korea_data/00/02/01/image
x_data: (1, 5, 100, 200, 3)
(8, 5, 100, 200, 3)
korea_data/00/02/02/image
x_data: (1, 5, 100, 200, 3)
(9, 5, 100, 200, 3)
korea_data/00/03/00/image
x_data: (1, 5, 100, 200, 3)
(10, 5, 100, 200, 3)
korea_data/00/03/01/image
x_data: (1, 5, 100, 200, 3)
(11, 5, 100, 200, 3)
korea_data/00/03/02/image
x_data: (1, 5, 100, 200, 3)
(12, 5, 100, 200, 3)
korea_data/00/04/00/image
x_data: (1, 5, 100, 200, 3)
(13, 5, 100, 200, 3)
korea_data/00/04/01/image
x_data: 

x_data: (1, 5, 100, 200, 3)
(110, 5, 100, 200, 3)
korea_data/01/08/02/image
x_data: (1, 5, 100, 200, 3)
(111, 5, 100, 200, 3)
korea_data/01/09/00/image
x_data: (1, 5, 100, 200, 3)
(112, 5, 100, 200, 3)
korea_data/01/09/01/image
x_data: (1, 5, 100, 200, 3)
(113, 5, 100, 200, 3)
korea_data/01/09/02/image
x_data: (1, 5, 100, 200, 3)
(114, 5, 100, 200, 3)
korea_data/01/10/00/image
x_data: (1, 5, 100, 200, 3)
(115, 5, 100, 200, 3)
korea_data/01/10/01/image
x_data: (1, 5, 100, 200, 3)
(116, 5, 100, 200, 3)
korea_data/01/10/02/image
x_data: (1, 5, 100, 200, 3)
(117, 5, 100, 200, 3)
korea_data/01/11/00/image
x_data: (1, 5, 100, 200, 3)
(118, 5, 100, 200, 3)
korea_data/01/11/01/image
x_data: (1, 5, 100, 200, 3)
(119, 5, 100, 200, 3)
korea_data/01/11/02/image
x_data: (1, 5, 100, 200, 3)
(120, 5, 100, 200, 3)
korea_data/01/12/00/image
x_data: (1, 5, 100, 200, 3)
(121, 5, 100, 200, 3)
korea_data/01/12/01/image
x_data: (1, 5, 100, 200, 3)
(122, 5, 100, 200, 3)
korea_data/01/12/02/image
x_data: (1, 

x_data: (1, 5, 100, 200, 3)
(218, 5, 100, 200, 3)
korea_data/02/16/02/image
x_data: (1, 5, 100, 200, 3)
(219, 5, 100, 200, 3)
korea_data/02/17/00/image
x_data: (1, 5, 100, 200, 3)
(220, 5, 100, 200, 3)
korea_data/02/17/01/image
x_data: (1, 5, 100, 200, 3)
(221, 5, 100, 200, 3)
korea_data/02/17/02/image
x_data: (1, 5, 100, 200, 3)
(222, 5, 100, 200, 3)
korea_data/02/18/00/image
x_data: (1, 5, 100, 200, 3)
(223, 5, 100, 200, 3)
korea_data/02/18/01/image
x_data: (1, 5, 100, 200, 3)
(224, 5, 100, 200, 3)
korea_data/02/18/02/image
x_data: (1, 5, 100, 200, 3)
(225, 5, 100, 200, 3)
korea_data/02/19/00/image
x_data: (1, 5, 100, 200, 3)
(226, 5, 100, 200, 3)
korea_data/02/19/01/image
x_data: (1, 5, 100, 200, 3)
(227, 5, 100, 200, 3)
korea_data/02/19/02/image
x_data: (1, 5, 100, 200, 3)
(228, 5, 100, 200, 3)
korea_data/02/20/00/image
x_data: (1, 5, 100, 200, 3)
(229, 5, 100, 200, 3)
korea_data/02/20/01/image
x_data: (1, 5, 100, 200, 3)
(230, 5, 100, 200, 3)
korea_data/02/20/02/image
x_data: (1, 

x_data: (1, 5, 100, 200, 3)
(326, 5, 100, 200, 3)
korea_data/03/24/02/image
x_data: (1, 5, 100, 200, 3)
(327, 5, 100, 200, 3)
korea_data/03/25/00/image
x_data: (1, 5, 100, 200, 3)
(328, 5, 100, 200, 3)
korea_data/03/25/01/image
x_data: (1, 5, 100, 200, 3)
(329, 5, 100, 200, 3)
korea_data/03/25/02/image
x_data: (1, 5, 100, 200, 3)
(330, 5, 100, 200, 3)
korea_data/03/26/00/image
x_data: (1, 5, 100, 200, 3)
(331, 5, 100, 200, 3)
korea_data/03/26/01/image
x_data: (1, 5, 100, 200, 3)
(332, 5, 100, 200, 3)
korea_data/03/26/02/image
x_data: (1, 5, 100, 200, 3)
(333, 5, 100, 200, 3)
korea_data/03/27/00/image
x_data: (1, 5, 100, 200, 3)
(334, 5, 100, 200, 3)
korea_data/03/27/01/image
x_data: (1, 5, 100, 200, 3)
(335, 5, 100, 200, 3)
korea_data/03/27/02/image
x_data: (1, 5, 100, 200, 3)
(336, 5, 100, 200, 3)
korea_data/04/00/00/image
x_data: (1, 5, 100, 200, 3)
(337, 5, 100, 200, 3)
korea_data/04/00/01/image
x_data: (1, 5, 100, 200, 3)
(338, 5, 100, 200, 3)
korea_data/04/00/02/image
x_data: (1, 

In [28]:
x_data_.shape

(420, 5, 100, 200, 3)

In [29]:
np.save('x_data.npy', x_data_)

In [30]:
x_data = np.load('x_data.npy') 

In [31]:
x_data.shape

(420, 5, 100, 200, 3)

## y_label

|모음|단어|코드|
|:---|:---|:---|
|ㅏ|하 가 사 아 다|00|
|ㅏㄴ|안|01|
|ㅏㅁ,ㅂ|감, 합|02|
|ㅓㅕ|녕 어|03|
|ㅗㅛ|도 조 송 좋 요|04| 
|ㅜㅠ|주 누 구|05|
|ㅣ|히 싫 니|06|
|ㅣㅁ|힘 심|07|
|ㅐㅔㅖ|내 계 세|08|
|ㅘ|와|09|
|ㅚ|죄|10|

In [74]:
data = [0]*15 + [1]*3 + [2]*6 + [3]*6 + [4]*15 + [5]*9 + [6]*9 + [7]*6 + [8]*9 + [9]*3 + [10]*3

In [75]:
data = np.repeat(data,5)

In [81]:
print(data.shape)
import keras
y_data = keras.utils.to_categorical(data, None) 
print(y_data.shape)

(420,)
(420, 11)


In [82]:
np.save('y_data.npy', y_data)

In [83]:
y_data = np.load('y_data.npy') 
y_data.shape

(420, 11)